In [89]:
import pandas as pd
import json

In [90]:
fec_data = pd.read_csv('data/FEC_data.csv')
FEC_subset_columns = ['year', 'CAND_ID', 'CAND_NAME', 'CAND_ICI', 'CAND_PTY_AFFILIATION', 'TTL_DISB', 'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT']

fec_data = fec_data[FEC_subset_columns]
fec_data.rename(columns={'CAND_ID': 'FEC_candidate_id'}, inplace=True)


In [91]:
house_data = pd.read_csv('data/1976-2022-house.csv')
drop_columns = ['state_cen', 'state_ic', 'version', 'fusion_ticket', 'writein', 'unofficial', 'mode', 'stage', 'state_fips', 'office']

house_data = house_data[(house_data['writein'] == False) & (house_data['unofficial'] == False) & (house_data['mode'] == 'TOTAL') & (house_data['stage'] == 'GEN')]

with pd.option_context('mode.chained_assignment', None):
    house_data.rename(columns={'candidate': 'candidate_name', 'candidatevotes': 'candidate_votes', 'totalvotes': 'total_votes'}, inplace=True)
    house_data.drop(columns=drop_columns, inplace=True)

# merging

In [92]:
from fuzzywuzzy import fuzz

# Function to find the best match between two name lists
def find_best_match(name, candidate_list):
    best_match = None
    best_ratio = 0
    for candidate in candidate_list:
        ratio = fuzz.token_sort_ratio(name, candidate)
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = candidate
    return best_match if best_ratio > 60 else None  # Return None if no good match found

In [93]:
pd.options.mode.chained_assignment = None
# Initialize an empty list to store all merged data
all_merged_data = []

for year in range(2010, 2022, 2):
    house_data_year = house_data[house_data['year'] == year]
    fec_data_year = fec_data[fec_data['year'] == year]

    # Ensure district is of the same type in both dataframes
    fec_data_year['CAND_OFFICE_DISTRICT'] = fec_data_year['CAND_OFFICE_DISTRICT'].astype(str)
    house_data_year['district'] = house_data_year['district'].astype(str)

    # Group finance data by state and district
    finance_grouped = fec_data_year.groupby(['CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT'])

    # Iterate through each district in house_data
    for (state, district), house_group in house_data_year.groupby(['state_po', 'district']):
        finance_group = finance_grouped.get_group((state, district)) if (state, district) in finance_grouped.groups else None
        finance_group_unk = finance_grouped.get_group((state, '-1')) if (state, '-1') in finance_grouped.groups else None
        
        if finance_group is not None or finance_group_unk is not None:
            house_candidates = house_group['candidate_name'].tolist()
            finance_candidates = []
            if finance_group is not None:
                finance_candidates.extend(finance_group['CAND_NAME'].tolist())
            if finance_group_unk is not None:
                finance_candidates.extend(finance_group_unk['CAND_NAME'].tolist())
            
            for _, house_row in house_group.iterrows():
                best_match = find_best_match(house_row['candidate_name'], finance_candidates)
                if best_match:
                    if best_match in finance_group['CAND_NAME'].tolist():
                        finance_row = finance_group[finance_group['CAND_NAME'] == best_match].iloc[0]
                    else:
                        finance_row = finance_group_unk[finance_group_unk['CAND_NAME'] == best_match].iloc[0]
                    merged_row = house_row.to_dict()
                    merged_row.update(finance_row.to_dict())
                    all_merged_data.append(merged_row)
                # If there's no match, append the row with only house data
                else:
                    merged_row = house_row.to_dict()
                    all_merged_data.append(merged_row)

# Create the final merged dataframe with all years
merged_df = pd.DataFrame(all_merged_data)
merged_df.drop(columns=['CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT'], inplace=True)

merged_df['district_id'] = merged_df['state_po'] + "_" + merged_df['district']
merged_df = merged_df[['district_id'] + [col for col in merged_df.columns if col != 'district_id']]

merged_df['race_id'] = merged_df['state_po'] + "_" + merged_df['district'] + "_" + merged_df['year'].astype(str)
merged_df = merged_df[['race_id'] + [col for col in merged_df.columns if col != 'race_id']]

# Save the merged dataset with all years to a single CSV file
merged_df = merged_df[merged_df['state'] != "DISTRICT OF COLUMBIA"]

# cleaning up parties we are not interested in
merged_df = merged_df.dropna(subset=['party'])
merged_df['party'] = merged_df['party'].apply(lambda x: 'Democrat' if x.lower() == 'democrat' else 'Republican' if x.lower() == 'republican' else 'Other')

In [94]:
# merged_df['TTL_DISB'].fillna(0, inplace=True)
# candidate id
merged_df['candidate_id'] = merged_df['year'].astype(str) + "_" + merged_df['FEC_candidate_id']

# Generate unique IDs for rows with NaN FEC_candidate_id
nan_mask = merged_df['candidate_id'].isna()
nan_count = nan_mask.sum()
unique_ids = [f'GEN_{i:06d}' for i in range(1, nan_count + 1)]

# Fill NaN values with unique IDs
merged_df.loc[nan_mask, 'candidate_id'] = unique_ids


In [95]:
# not interested in DC for this analysis
merged_df = merged_df[merged_df['state'] != "DISTRICT OF COLUMBIA"]

In [96]:
# need to merge the same candidates with different parties together
merged_df[merged_df['candidate_id']=="2018_H2NY03089"]

,race_id,district_id,year,state,state_po,district,runoff,special,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB,candidate_id
5918,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,12504,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5919,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,2535,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5920,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,474,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5921,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Republican,112565,241217,H2NY03089,I,3183657.13,2018_H2NY03089
5922,NY_2_2018,NY_2,2018,NEW YORK,NY,2,NaN,False,PETER T KING,Other,0,241217,H2NY03089,I,3183657.13,2018_H2NY03089


In [97]:
# need to handle candidates that are miscategorized in the FEC data as having no district like "STEVE LINDBECK"
# need to handle candidates that raised no money
merged_df[merged_df['race_id']=='AK_0_2016']

,race_id,district_id,year,state,state_po,district,runoff,special,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB,candidate_id
3903,AK_0_2016,AK_0,2016,ALASKA,AK,0,NaN,False,BERNIE SOUPHANAVONG,Other,9093,308198,NaN,NaN,NaN,GEN_001018
3904,AK_0_2016,AK_0,2016,ALASKA,AK,0,NaN,False,DON YOUNG,Republican,155088,308198,H6AK00045,I,1322055.12,2016_H6AK00045
3905,AK_0_2016,AK_0,2016,ALASKA,AK,0,NaN,False,JIM C MCDERMOTT,Other,31770,308198,NaN,NaN,NaN,GEN_001019
3906,AK_0_2016,AK_0,2016,ALASKA,AK,0,NaN,False,STEVE LINDBECK,Democrat,111019,308198,H6AK00235,NaN,1098098.09,2016_H6AK00235


In [98]:
# Group by candidate_id and aggregate
grouped_df = merged_df.groupby('candidate_id').agg({
    'race_id': 'first',
    'district_id': 'first',
    'year': 'first',
    'state': 'first',
    'state_po': 'first',
    'district': 'first',
    'runoff': 'first',
    'special': 'first',
    'candidate_name': 'first',
    'party': lambda x: 'Democrat' if 'Democrat' in x.values else ('Republican' if 'Republican' in x.values else 'Other'),
    'candidate_votes': 'sum',
    'total_votes': 'first',
    'FEC_candidate_id': 'first',
    'CAND_ICI': 'first',
    'TTL_DISB': 'first'
}).reset_index()

grouped_df = grouped_df[(grouped_df['runoff'] != True)]
grouped_df = grouped_df[grouped_df['special'] != True]

grouped_df.drop(columns=['state_po', 'runoff', 'special'], inplace=True)


In [99]:
grouped_df.sort_values(by=['race_id'], inplace=True)
grouped_df.to_csv('data/cleaned/merged_house_finance_data_all_years.csv', index=False)

In [100]:
grouped_df

,candidate_id,race_id,district_id,year,state,district,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB
0,2010_H0AK00089,AK_0_2010,AK_0,2010,ALASKA,0,HARRY T CRAWFORD JR,Democrat,77606,254335,H0AK00089,C,235571.43
636,2010_H6AK00045,AK_0_2010,AK_0,2010,ALASKA,0,DON YOUNG,Republican,175384,254335,H6AK00045,I,887310.33
1081,2012_H2AK00119,AK_0_2012,AK_0,2012,ALASKA,0,SHARON M CISSNA,Democrat,82927,289804,H2AK00119,C,24388.00
1569,2012_H6AK00045,AK_0_2012,AK_0,2012,ALASKA,0,DON YOUNG,Republican,185296,289804,H6AK00045,I,665974.39
5363,GEN_000372,AK_0_2012,AK_0,2012,ALASKA,0,TED GIANOUTSOS,Other,5589,289804,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,2018_H2WY00133,WY_0_2018,WY_0,2018,WYOMING,0,DANIEL CLYDE CUMMINGS,Other,6070,201245,H2WY00133,C,485.00
6962,GEN_001971,WY_0_2020,WY_0,2020,WYOMING,0,JEFF HAGGIT,Other,7905,278503,None,None,NaN
4784,2020_H6WY00159,WY_0_2020,WY_0,2020,WYOMING,0,LIZ CHENEY,Republican,185732,278503,H6WY00159,I,3066534.91
4553,2020_H0WY01036,WY_0_2020,WY_0,2020,WYOMING,0,LYNNETTE GREY BULL,Democrat,66576,278503,H0WY01036,C,132285.65


# switching to race as row

In [101]:
# Group by race_id
grouped = grouped_df.groupby('race_id')

# Filter out races with only one candidate
multi_candidate_races = grouped.filter(lambda x: len(x) > 1)

# Regroup after filtering
grouped = multi_candidate_races.groupby('race_id')

# Create a new DataFrame with the desired columns
race_df = pd.DataFrame({
    'district_id': grouped['district_id'].first(),
    'year': grouped['year'].first(),
    'district': grouped['district'].first(),
    'state': grouped['state'].first(),
    'total_votes': grouped['total_votes'].first(),
})

# Find the winner for each race
def get_winner_info(group):
    winner = group.loc[group['candidate_votes'].idxmax()]
    runner_up = group.nlargest(2, 'candidate_votes').iloc[1]
    return pd.Series({
        'winner_name': winner['candidate_name'],
        'winner_party': winner['party'],
        'winner_CAND_ICI': winner['CAND_ICI'],
        'runner_up_name': runner_up['candidate_name'],
        'runner_up_party': runner_up['party'],
        'runner_up_CAND_ICI': runner_up['CAND_ICI'],
        'margin_of_victory': round(100 * (winner['candidate_votes'] - runner_up['candidate_votes']) / group['total_votes'].iloc[0], 2),
        'spending_winner': winner['TTL_DISB'],
        'spending_runner_up': runner_up['TTL_DISB']
    })

# Apply the function to get winner information
winner_info = grouped.apply(get_winner_info)

# Combine the basic race information with the winner information
race_df = race_df.join(winner_info)

# Reset the index to make race_id a column
race_df = race_df.reset_index()

/tmp/ipykernel_6167/2850129064.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  winner_info = grouped.apply(get_winner_info)


In [102]:
# Fill in missing values based on the rule: if one is "C", the other must be "I"
race_df.loc[race_df['winner_CAND_ICI'] == 'C', 'runner_up_CAND_ICI'] = race_df.loc[race_df['winner_CAND_ICI'] == 'C', 'runner_up_CAND_ICI'].fillna('I')
race_df.loc[race_df['runner_up_CAND_ICI'] == 'C', 'winner_CAND_ICI'] = race_df.loc[race_df['runner_up_CAND_ICI'] == 'C', 'winner_CAND_ICI'].fillna('I')

# Fix races that are not really open
race_df.loc[(race_df['winner_CAND_ICI'] == 'O') & (race_df['runner_up_CAND_ICI'] == 'C'), 'winner_CAND_ICI'] = 'I'
race_df.loc[(race_df['winner_CAND_ICI'] == 'O') & (race_df['runner_up_CAND_ICI'] == 'I'), 'winner_CAND_ICI'] = 'C'
race_df.loc[(race_df['runner_up_CAND_ICI'] == 'O') & (race_df['winner_CAND_ICI'] == 'C'), 'runner_up_CAND_ICI'] = 'I'
race_df.loc[(race_df['runner_up_CAND_ICI'] == 'O') & (race_df['winner_CAND_ICI'] == 'I'), 'runner_up_CAND_ICI'] = 'C'

# double incumbent races
race_df.loc[race_df["winner_CAND_ICI"].isin(["I"]) & (race_df["runner_up_CAND_ICI"] == "I"), 'runner_up_CAND_ICI'] = "C"

# when both are challengers, set the winner to incumbent
race_df.loc[(race_df["winner_CAND_ICI"] == "C") & (race_df["runner_up_CAND_ICI"] == "C"), 'winner_CAND_ICI'] = "I"


In [103]:
with open('data/wiki/wiki_scrape_pvi.json', 'r') as file:
    pvi_data = json.load(file)

def convert_pvi(pvi):
    if 'R+' in pvi:
        return int(pvi.replace('R+', ''))
    elif 'D+' in pvi:
        return -int(pvi.replace('D+', ''))
    elif 'even' in pvi.lower():
        return 0
    else:
        print(f"PVI value not recognized: {pvi}")
        return None


for year, state_data in pvi_data.items():
    for state, districts in state_data.items():
        for district_info in districts:
            district_name = district_info['district']
            pvi = district_info['pvi']
            
            # Extract district number
            if 'at-large' in district_name:
                district_number = 0
            elif " " in district_name:
                district_number = district_name.split(' ')[-1]
                if "\xa0" in district_name:
                    district_number = int(district_name.split('\xa0')[-1])
                else:
                    district_number = int(district_name.split(' ')[-1])
            else:
                district_number = int(district_name.split('\u00a0')[-1])
            
            # Update merged_df with PVI information
            mask = (
                (race_df['year'] == int(year)) &
                (race_df['state'] == state.upper()) &
                (race_df['district'] == str(district_number))
            )
            converted_pvi = convert_pvi(pvi)
            race_df.loc[mask, 'pvi'] = converted_pvi

race_df.drop(columns=['state', 'district'], inplace=True)


In [104]:
race_df

,race_id,district_id,year,total_votes,winner_name,winner_party,winner_CAND_ICI,runner_up_name,runner_up_party,runner_up_CAND_ICI,margin_of_victory,spending_winner,spending_runner_up,pvi
0,AK_0_2010,AK_0,2010,254335,DON YOUNG,Republican,I,HARRY T CRAWFORD JR,Democrat,C,38.44,887310.33,235571.43,13.0
1,AK_0_2012,AK_0,2012,289804,DON YOUNG,Republican,I,SHARON M CISSNA,Democrat,C,35.32,665974.39,24388.00,13.0
2,AK_0_2014,AK_0,2014,279741,DON YOUNG,Republican,I,FORREST DUNBAR,Democrat,C,10.00,809039.20,230088.83,12.0
3,AK_0_2016,AK_0,2016,308198,DON YOUNG,Republican,I,STEVE LINDBECK,Democrat,None,14.30,1322055.12,1098098.09,12.0
4,AK_0_2018,AK_0,2018,282166,DON YOUNG,Republican,I,ALYSE S GALVIN,Democrat,C,6.58,1387687.05,1943398.59,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,WY_0_2012,WY_0,2012,250700,CYNTHIA M LUMMIS,Republican,I,CHRIS HENRICHSEN,Democrat,None,43.43,631025.66,NaN,20.0
2476,WY_0_2014,WY_0,2014,171153,CYNTHIA M LUMMIS,Republican,I,RICHARD GRAYSON,Democrat,None,43.96,300949.02,NaN,22.0
2477,WY_0_2016,WY_0,2016,258788,LIZ CHENEY,Republican,O,RYAN GREENE,Democrat,O,31.19,2148255.72,186092.78,22.0
2478,WY_0_2018,WY_0,2018,201245,LIZ CHENEY,Republican,I,GREG HUNTER,Democrat,C,33.82,715739.91,47069.51,25.0


In [105]:
with open('data/wiki/wiki_scrape_nhe.json', 'r') as file:
    nhe_data = json.load(file)
    
for year, data in nhe_data.items():
    republican_percentage = 0
    democratic_percentage = 0
    
    for party_data in data:
        if party_data['party'] == 'Republican Party':
            republican_percentage = party_data['percentage']
        elif party_data['party'] == 'Democratic Party':
            democratic_percentage = party_data['percentage']
    percent_difference = republican_percentage - democratic_percentage
    
    # Update the 'NHE' column for the corresponding year
    race_df.loc[race_df['year'] == int(year), 'NHE'] = round(percent_difference, 2)
    
    # Print the year and the difference
    print(f"Year: {year}, Percent difference between Republican and Democratic votes: {percent_difference:.2f}%")

Year: 2022, Percent difference between Republican and Democratic votes: 2.72%
Year: 2020, Percent difference between Republican and Democratic votes: -3.10%
Year: 2018, Percent difference between Republican and Democratic votes: -8.60%
Year: 2016, Percent difference between Republican and Democratic votes: 1.10%
Year: 2014, Percent difference between Republican and Democratic votes: 5.70%
Year: 2012, Percent difference between Republican and Democratic votes: -1.10%
Year: 2010, Percent difference between Republican and Democratic votes: 6.80%
Year: 2008, Percent difference between Republican and Democratic votes: -10.60%


In [106]:
race_df.to_csv('data/cleaned/races.csv', index=False)